In [5]:
from pywgraph import WeightedDirectedGraph, WeightedDirectedEdge, Group
import numpy as np 

In [6]:
r3_group = Group("R^3", np.ones(3), lambda x, y: x * y, lambda x, y: x / y, lambda x: hash(tuple(x)))
graph = WeightedDirectedGraph(
    {"EUR", "USD", "GBP"},
    {
        WeightedDirectedEdge("EUR", "USD", np.array([1.1, 1.2, 1.3]), r3_group),
        WeightedDirectedEdge("USD", "GBP", np.array([0.9, 1.0, 0.8]), r3_group),
    },
    r3_group
)

In [9]:
graph.path_weight(["EUR", "USD", "GBP"])

array([0.99, 1.2 , 1.04])